In [2]:
import pandas as pd
import numpy as np
import math
import statistics
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, f1_score, auc

In [206]:
df = pd.read_csv("apartments_for_rent_classified_10K.csv", on_bad_lines='skip', encoding ='cp1252', delimiter=';')


In [207]:
df.head(1)

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668626895,housing/rent/apartment,"Studio apartment 2nd St NE, Uhland Terrace NE,...","This unit is located at second St NE, Uhland T...",NaN,NaN,0.0,USD,No,Thumbnail,...,$790,Monthly,101,NaN,Washington,DC,38.9057,-76.9861,RentLingo,1577359415


In [208]:
df = df.drop('id', axis=1)
df = df.drop('category', axis=1)
df = df.drop('title', axis=1)
df = df.drop('body', axis=1)
df = df.drop('amenities', axis=1)
df = df.drop('fee', axis=1)
df = df.drop('has_photo', axis=1)
df = df.drop('time', axis=1)
df = df.drop('source', axis=1)
df = df.drop('latitude', axis=1)
df = df.drop('longitude', axis=1)
df = df.drop('cityname', axis=1)
df = df.drop('address', axis=1)
df = df.drop('price_type', axis=1)
df = df.drop('price_display', axis=1)
df = df.drop('currency', axis=1)

In [209]:
df.head()

,bathrooms,bedrooms,pets_allowed,price,square_feet,state
0,NaN,0.0,NaN,790,101,DC
1,NaN,1.0,NaN,425,106,IN
2,1.0,0.0,NaN,1390,107,VA
3,1.0,0.0,NaN,925,116,WA
4,NaN,0.0,NaN,880,125,VA


In [210]:
dfx = pd.get_dummies(df[['state', 'bathrooms', 'bedrooms', 'price', 'square_feet']])
dfy = df[['pets_allowed']]

In [211]:
dfx.head()

,bathrooms,bedrooms,price,square_feet,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,NaN,0.0,790,101,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,NaN,1.0,425,106,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1.0,0.0,1390,107,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1.0,0.0,925,116,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,NaN,0.0,880,125,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [212]:
dfy.head()

,pets_allowed
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [213]:
dfy['pets_allowed'].value_counts()

pets_allowed
Cats,Dogs    5228
Cats          485
Dogs          124
Name: count, dtype: int64

In [214]:
dfy = dfy["pets_allowed"].replace(np.nan, 'No Pets Allowed')

In [215]:
dfy = np.array(dfy)

In [216]:
dfy = pd.DataFrame(dfy, columns =['Pets Allowed']) 

In [217]:
dfy.head(2)

,Pets Allowed
0,No Pets Allowed
1,No Pets Allowed


In [218]:
pd.set_option("display.max_rows", None)

In [219]:
dfx.head()

,bathrooms,bedrooms,price,square_feet,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,NaN,0.0,790,101,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,NaN,1.0,425,106,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1.0,0.0,1390,107,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1.0,0.0,925,116,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,NaN,0.0,880,125,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [220]:
#dfx['bathrooms'].isnull().sum

In [221]:
dfx['bathrooms'] = dfx['bathrooms'].fillna(1)
dfx['bedrooms'] = dfx['bedrooms'].fillna(1)

In [222]:
dfx.head()

,bathrooms,bedrooms,price,square_feet,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,1.0,0.0,790,101,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1.0,1.0,425,106,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1.0,0.0,1390,107,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1.0,0.0,925,116,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,1.0,0.0,880,125,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [223]:
dfx['bedrooms'].isna().sum()

np.int64(0)

In [226]:
dfy2 = np.where(dfy["Pets Allowed"] == "No Pets Allowed", "No", "Yes")
                                        

In [227]:
dfy2 = pd.DataFrame(dfy2, columns =['Pets Allowed']) 

In [228]:
dfy2.head()

,Pets Allowed
0,No
1,No
2,No
3,No
4,No


In [229]:
dfy2['Pets Allowed'].value_counts()

Pets Allowed
Yes    5837
No     4163
Name: count, dtype: int64

In [230]:
dfx.head()

,bathrooms,bedrooms,price,square_feet,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,1.0,0.0,790,101,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1.0,1.0,425,106,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1.0,0.0,1390,107,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1.0,0.0,925,116,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,1.0,0.0,880,125,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [231]:
dfy2['Pets Allowed'].value_counts()

Pets Allowed
Yes    5837
No     4163
Name: count, dtype: int64

In [232]:
dfy2.head()

,Pets Allowed
0,No
1,No
2,No
3,No
4,No


In [233]:
dfx.to_csv('Apartment_Factors.csv', index=False)
dfy.to_csv('Pets Allowed1.csv', index=False)
dfy2.to_csv('Pets Allowed2.csv', index=False)